In [1]:
!nvidia-smi

Thu May  4 22:41:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/Omid-Nejati/MedViT.git

Cloning into 'MedViT'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 34 (delta 6), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (34/34), 756.87 KiB | 9.83 MiB/s, done.


In [2]:
cd /content/MedViT

/content/MedViT


In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchsummary import summary


##Model

In [4]:
!pip install timm
!pip install einops

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.3 MB ? eta -:--:--
   ------------- -------------------------- 0.8/2.3 MB 1.3 MB/s eta 0:00:02
   ------------------ --------------------- 1.0/2.3 MB 1.3 MB/s eta 0:00:01
   ---------------------- ----------------- 1.3/2.3 MB 1.4 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.3 MB 1.5 MB/s eta 0:00:01
   ------------------------------------ --- 2.1/2.3 MB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 2.3/2.3 MB 1.5 MB/s eta 0:00:00


In [33]:
from MedViT import MedViT_small as tiny

In [34]:
model = tiny()

initialize_weights...


In [35]:
model.proj_head[0]

Linear(in_features=1024, out_features=1000, bias=True)

In [36]:
model.proj_head[0] = torch.nn.Linear(in_features=1024, out_features=14, bias=True)

In [9]:
model = model.cuda()

##Dataset

In [9]:
!pip install medmnist

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114263 sha256=d2cfebe274982f4ca8e5c7dfd2e9e694fa92fb984c192f4260bcf97cbaf6812f
  Stored in directory: c:\users\rrahman3\appdata\local\pip\cache\wheels\9e\5b\45\29f72e55d87a29426b04b3cfdf20325c079eb97ab74f59017d
Successfully built fire


In [10]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [28]:
data_flag = 'breastmnist'
data_flag = 'chestmnist'
# [tissuemnist, pathmnist, chestmnist, dermamnist, octmnist, 
# pnemoniamnist, retinamnist, breastmnist, bloodmnist, tissuemnist, organamnist, organcmnist, organsmnist]
download = True

NUM_EPOCHS = 10
BATCH_SIZE = 10
lr = 0.005

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [37]:
info

{'python_class': 'ChestMNIST',
 'description': 'The ChestMNIST is based on the NIH-ChestXray14 dataset, a dataset comprising 112,120 frontal-view X-Ray images of 30,805 unique patients with the text-mined 14 disease labels, which could be formulized as a multi-label binary-class classification task. We use the official data split, and resize the source images of 1×1024×1024 into 1×28×28.',
 'url': 'https://zenodo.org/records/10519652/files/chestmnist.npz?download=1',
 'MD5': '02c8a6516a18b556561a56cbdd36c4a8',
 'url_64': 'https://zenodo.org/records/10519652/files/chestmnist_64.npz?download=1',
 'MD5_64': '9de6cd0b934ebb5b7426cfba5efbae16',
 'url_128': 'https://zenodo.org/records/10519652/files/chestmnist_128.npz?download=1',
 'MD5_128': 'db107e5590b27930b62dbaf558aebee3',
 'url_224': 'https://zenodo.org/records/10519652/files/chestmnist_224.npz?download=1',
 'MD5_224': '45bd33e6f06c3e8cdb481c74a89152aa',
 'task': 'multi-label, binary-class',
 'label': {'0': 'atelectasis',
  '1': 'cardi

In [29]:
from torchvision.transforms.transforms import Resize
# preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    torchvision.transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.Lambda(lambda image: image.convert('RGB')),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=download)
test_dataset = DataClass(split='test', transform=test_transform, download=download)

# pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

100%|██████████| 82802576/82802576 [00:22<00:00, 3711103.31it/s] 


Using downloaded and verified file: C:\Users\rrahman3\.medmnist\chestmnist.npz


In [30]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset ChestMNIST of size 28 (chestmnist)
    Number of datapoints: 78468
    Root location: C:\Users\rrahman3\.medmnist
    Split: train
    Task: multi-label, binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'atelectasis', '1': 'cardiomegaly', '2': 'effusion', '3': 'infiltration', '4': 'mass', '5': 'nodule', '6': 'pneumonia', '7': 'pneumothorax', '8': 'consolidation', '9': 'edema', '10': 'emphysema', '11': 'fibrosis', '12': 'pleural', '13': 'hernia'}
    Number of samples: {'train': 78468, 'val': 11219, 'test': 22433}
    Description: The ChestMNIST is based on the NIH-ChestXray14 dataset, a dataset comprising 112,120 frontal-view X-Ray images of 30,805 unique patients with the text-mined 14 disease labels, which could be formulized as a multi-label binary-class classification task. We use the official data split, and resize the source images of 1×1024×1024 into 1×28×28.
    License: CC BY 4.0
Dataset ChestMNIST of size 28 (chestmnist)
    Number of datapoints: 22

In [32]:
from PIL import Image
for x, y in train_loader:
    # print(x, y)
    print(x.shape, y.shape)
    output_numpy = x[0].permute(1, 2, 0).cpu().numpy()  # Convert from (C, H, W) to (H, W, C)

    # Convert the NumPy array to an image (ensure the values are in [0, 255] for uint8 type)
    # output_image = Image.fromarray((output_numpy * 255).astype('uint8'))
    # obj = obj.transpose((0, 2, 3, 1))
    output_numpy = np.clip(((output_numpy + 1) / 2.0) * 256, 0, 255)

    # Show the image
    print(output_numpy.shape)
    output_numpy = np.asarray(np.uint8(output_numpy), dtype=np.uint8)
    output_image = Image.fromarray(output_numpy, mode="RGB")
    output_image.show()
    break

torch.Size([10, 3, 224, 224]) torch.Size([10, 14])
(224, 224, 3)


## Train

In [38]:
# define loss function and optimizer
if task == "multi-label, binary-class":
    criterion = nn.BCEWithLogitsLoss()
else:
    criterion = nn.CrossEntropyLoss()
    
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [40]:
for inputs, targets in tqdm(train_loader):
        # inputs, targets = inputs.cuda(), targets.cuda()
    print(targets.shape)
    print(targets)
    print(inputs.shape)
    print(inputs)
    break


  0%|          | 0/7847 [00:00<?, ?it/s]

torch.Size([10, 14])
tensor([[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=torch.int32)
torch.Size([10, 3, 224, 224])
tensor([[[[-0.8667, -0.8667, -0.8667,  ..., -0.6392, -0.6392, -0.6392],
          [-0.8667, -0.8667, -0.8667,  ..., -0.6392, -0.6392, -0.6392],
          [-0.8667, -0.8667, -0.8667,  ..., -0.6392, -0.6392, -0.6392],
          ...,
          [-1.0000, -1.0000, -1.0000,  ...,  0.1922,  0.1922,  0.1922],
          [-1.0000, -1.0000, -1.0000,  ...,  0.1922,  0.1922,  0.1922],
          [-1.0000, -1.0000, -1.00

  0%|          | 0/7847 [00:01<?, ?it/s]


In [39]:
# train

for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    print('Epoch [%d/%d]'% (epoch+1, NUM_EPOCHS))
    model.train()
    for inputs, targets in tqdm(train_loader):
        # inputs, targets = inputs.cuda(), targets.cuda()
        # forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(inputs)
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
            loss = criterion(outputs, targets)
        else:
            targets = targets.squeeze().long()
            loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()

Epoch [1/10]


 60%|█████▉    | 4686/7847 [40:02:44<27:00:47, 30.76s/it]


KeyboardInterrupt: 

##Test

In [20]:
# evaluation

def test(split):
    model.eval()
    y_true = torch.tensor([])
    y_score = torch.tensor([])
    
    data_loader = train_loader_at_eval if split == 'train' else test_loader

    with torch.no_grad():
        for inputs, targets in data_loader:
            # inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
                outputs = outputs.softmax(dim=-1)
            else:
                targets = targets.squeeze().long()
                outputs = outputs.softmax(dim=-1)
                targets = targets.float().resize_(len(targets), 1)

            y_true = torch.cat((y_true, targets), 0)
            y_score = torch.cat((y_score, outputs), 0)

        y_true = y_true.cpu().numpy()
        y_score = y_score.detach().cpu().numpy()
        
        evaluator = Evaluator(data_flag, split)
        metrics = evaluator.evaluate(y_score)
    
        print('%s  auc: %.3f  acc:%.3f' % (split, *metrics))

        
print('==> Evaluating ...')
test('train')
test('test')

==> Evaluating ...
train  auc: 0.839  acc:0.661
test  auc: 0.788  acc:0.654
